# Depictio Component Export API Demo

This notebook demonstrates how to **export rendered dashboard components** from Depictio
and reconstruct them in a standalone Python environment — no Depictio UI required.

We use the `/render` API endpoint to fetch:
1. **Figure** — a Plotly scatter plot, rendered as a standalone interactive figure
2. **Card** — a computed aggregation value (average sepal length)
3. **Table** — raw tabular data loaded into a pandas DataFrame

All examples target the **Iris Dashboard demo** (`6824cb3b89d2b72169309737`).

In [ ]:
import requests

API_BASE = "http://localhost:8135/depictio/api/v1"
DASHBOARD_ID = "6824cb3b89d2b72169309737"
TOKEN = (
    "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9."
    "eyJuYW1lIjoiZGVmYXVsdF90b2tlbiIsInRva2VuX2xpZmV0aW1lIjoibG9uZy1saXZlZCIsInRva2VuX3R5cGUiOiJiZWFyZXIiLCJzdWIiOiI2NzY1OGJhMDMzYzhiNTlhZDQ4OWQ3YzciLCJleHAiOjE4MDM3MjczMjR9."
    "kXyOArNVvh9UMjcNRchdd6bhbV6TivUskevm-1X7IZw2UtSjdysxq4V7e0w4NC2CRtPoX7fiSxoRh172-X8J0fEc1wlX8432zBq1o64HMExED9vUku1ubcamxDIq3QVaVo25pq4qiVWpwhMVhf3TI5O7h1ZJCXflpyWTh11-0rZ5TeObf-i1pJ6Z-Wa2p4pPf5MkHufm2U0iFL6Ce3QaXqX4dA-LxZov1FSR5rKru7axHL_0pTk9uTyoBLsMTN7uG6jxUOBQfb29yy9zob5f8gS9MAfgd64A7UvOGi0_hXs_sQirf_2YRrA30k_0Yz2ygpRD9JJJHNHVkqnFBRFzOA"
)

HEADERS = {"Authorization": f"Bearer {TOKEN}", "Content-Type": "application/json"}


def render_component(tag: str, theme: str = "light", timeout: int = 30, **kwargs) -> dict:
    """Call the Depictio render endpoint and return the JSON response."""
    url = f"{API_BASE}/render/{DASHBOARD_ID}/components/{tag}"
    payload = {"theme": theme, "timeout": timeout, **kwargs}
    resp = requests.post(url, json=payload, headers=HEADERS)
    resp.raise_for_status()
    return resp.json()

---
## Example 1: Export a Plotly Figure

Render the **scatter plot** (sepal length vs. sepal width, colored by variety) and
display it as a standalone Plotly figure — identical to what Depictio shows, but
running entirely outside the platform.

In [ ]:
import plotly.graph_objects as go

# Render the scatter figure via API (identified by its tag)
result = render_component("figure-scatter_sepal_length_sepal_width-d4feb7", theme="light")
print(f"Status: {result['status']}")
print(f"Component type: {result['component_type']}")
print(f"Component tag: {result['component_tag']}")

# Reconstruct the Plotly figure from the exported JSON
fig = go.Figure(result["data"]["figure"])
fig.update_layout(title_text="Exported from Depictio — Sepal Length vs Width")
fig.show()

In [ ]:
# The data_info field tells us about sampling and row counts
print("Data info:", result["data"].get("data_info", {}))

---
## Example 2: Export a Card (Computed Aggregation)

Render the **average sepal length** card. This returns a single computed value
that can be used in reports, alerts, or downstream pipelines.

In [ ]:
result = render_component("card-sepal_length_average-cdb8b8")
print(f"Status: {result['status']}")
print(f"Component type: {result['component_type']}")
print(f"Component tag: {result['component_tag']}")

card = result["data"]
print(f"\n--- Exported Card ---")
print(f"Column:      {card['column']}")
print(f"Aggregation: {card['aggregation']}")
print(f"Value:       {card['value']}")

In [ ]:
# Use the value programmatically — e.g. in a threshold check
threshold = 6.0
avg = card["value"]
status = "ABOVE" if avg >= threshold else "BELOW"
print(f"Average sepal length ({avg:.2f}) is {status} threshold ({threshold})")

---
## Example 3: Export a Table as a DataFrame

Render the **data table** component and load it directly into pandas for
analysis, joining with other data, or exporting to CSV/Excel.

In [ ]:
import pandas as pd

result = render_component("table-table-822ba8")
print(f"Status: {result['status']}")
print(f"Component type: {result['component_type']}")
print(f"Total rows in source: {result['data']['total_rows']}")
print(f"Rows returned:        {result['data']['returned_rows']}")

# Load into pandas
df = pd.DataFrame(result["data"]["rows"], columns=result["data"]["columns"])
df.head(10)

In [ ]:
# Quick analysis on the exported data
print(df.describe())
print(f"\nVarieties: {df['variety'].unique().tolist() if 'variety' in df.columns else 'N/A'}")

In [ ]:
# Export to CSV — the data now lives completely outside Depictio
df.to_csv("iris_exported.csv", index=False)
print("Saved to iris_exported.csv")

---
## Summary

| Component | Tag | What you get |
|-----------|-----|--------------|
| **Figure** | `figure-scatter_sepal_length_sepal_width-d4feb7` | Full Plotly JSON — `go.Figure(data["figure"])` |
| **Card** | `card-sepal_length_average-cdb8b8` | Scalar value + aggregation metadata |
| **Table** | `table-table-822ba8` | Row data + column list — `pd.DataFrame(data["rows"])` |

The render API lets you programmatically extract any dashboard component and use it
in notebooks, scripts, reports, or downstream pipelines — without ever opening the Depictio UI.